# Cobwebbing

Adam Rumpf

Created 4/20/21

Based on a <a href="https://github.com/adam-rumpf/mathematica-class-demonstrations#cobwebbing" target="_blank">Mathematica class demonstration</a>.

This is a standalone widget for playing around with cobweb diagrams for various dynamical systems. See the full notebook [here](./cobwebbing.ipynb).

[Main Project Page](.././index.ipynb)

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Define parameters
COBWEB_MAX = 20 # number of cobweb iterations to generate

# Define functions

def lmap(x, lim=1.0, r=1.0, h=0.0):
    """Discrete logistic map with absolute harvesting.
    
    Positional arguments:
    x - input value
    
    Keyword arguments:
    lim (1.0) - population limit
    r (1.0) - intrinsic growth rate
    """
    
    return x + r*x*(1 - (x/lim)) - h

def tmap(x, lb=0.5, ub=1.0, r=1.0):
    """Discrete population growth with upper and lower stable population bounds.
    
    Positional arguments:
    x - input value
    
    Keyword arguments:
    lb (0.5) - lower population limit
    ub (1.0) - upper population limit
    r (1.0) - intrinsic growth rate
    """
    
    return x + r*x*(x-lb)*(ub-x)

def mmap(x, r=0.0):
    """Discrete map meant to have an adjustable slope at one of its equlibria.
    
    Positional arguments:
    x - input value
    
    Keyword arguments:
    r (0.0) - slope at intermediate equilibrium point (<= 1.5)
    """
    
    a = 4 - 4*r
    return (a*x*x - 1.5*a*x + (1.0 + 0.5*a))*x

def cobweb_update(x0, cwx, cwy, lim=1.0, r=1.0, mode=0):
    """Updates the global cobweb lists. Lists are edited in-place.
    
    Positional arguments:
    x0 - initial population value
    cwx - reference to a list of cobweb plot x-values
    cwy - reference to a list of cobweb plot y-values
    
    Keyword arguments:
    lim (1.0) - population limit
    r (1.0) - intrinsic growth rate
    mode (0) - 0 for logistic with harvesting, 1 for bounded population, 2 for slope map
    """
    
    # Generate cobweb coordinates
    cwx[0] = x0
    cwy[0] = 0.0
    for i in range(0, 2*COBWEB_MAX, 2):
        cwx[i+1] = cwx[i]
        if mode == 0:
            cwy[i+1] = max(lmap(cwx[i], r=r), 0.0)
        elif mode == 1:
            cwy[i+1] = max(tmap(cwx[i], r=r), 0.0)
        elif mode == 2:
            cwy[i+1] = max(mmap(cwx[i], r=r), 0.0)
        cwx[i+2] = cwy[i+1]
        cwy[i+2] = cwy[i+1]
    cwx[-1] = cwx[-2]
    if mode == 0:
        cwy[-1] = max(lmap(cwx[-1], r=r), 0.0)
    elif mode == 1:
        cwy[-1] = max(tmap(cwx[-1], r=r), 0.0)
    elif mode == 2:
        cwy[-1] = max(mmap(cwx[-1], r=r), 0.0)

# Generate x- and n-values
x = np.linspace(0, 1.5, 101)
nval = [np.floor((n+1)/2) for n in range(2*COBWEB_MAX+2)]

# Set up side-by-side plots and initialize cobweb coordinate lists
figs1, ax1 = plt.subplots(1, 2, figsize=(10, 4))
cwx1 = np.zeros(2*COBWEB_MAX+2) # cobweb x-coordinates
cwy1 = np.zeros_like(cwx1) # cobweb y-coordinates

# Draw plot lines
@widgets.interact(step=(0, 2*COBWEB_MAX, 1), r=(0.5, 3.0, 0.01), x0=(0.0, 1.25, 0.01))
def update1(step=4, r=1.5, x0=0.25):
    global ax1, cwx1, cwy1
    
    # Cobweb plot
    ax1[0].clear()
    ax1[0].set_xlim([0, 1.5])
    ax1[0].set_ylim([0, 1.5])
    ax1[0].grid(False)
    ax1[0].set_title("Cobweb Plot")
    ax1[0].set_xlabel("$x_n$")
    ax1[0].set_ylabel("$x_{n+1}$")
    cobweb_update(x0, cwx1, cwy1, r=r, mode=0)
    ax1[0].plot(x, lmap(x, r=r), color="C0")
    ax1[0].plot(x, x, color="black")
    ax1[0].plot(cwx1[:step+2], cwy1[:step+2], color="C1")
    ax1[0].plot(cwx1[step:step+2], cwy1[step:step+2], color="red")
    
    # Scatter plot
    ax1[1].clear()
    ax1[1].set_ylim([0, 1.5])
    ax1[1].grid(False)
    ax1[1].set_title("Time Series")
    ax1[1].set_xlabel("$n$")
    ax1[1].set_ylabel("$x_n$")
    ax1[1].plot(np.append([0], nval[1:step+2:2]), np.append([x0], cwy1[1:step+2:2]), color="C0", marker=".", markersize=10)
    ax1[1].plot(nval[step+1], cwy1[step+1], color="red", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=4, description='step', max=40), FloatSlider(value=1.5, description='r', …

In [2]:
# Set up side-by-side plots and initialize cobweb coordinate lists
figs2, ax2 = plt.subplots(1, 2, figsize=(10, 4))
cwx2 = np.zeros(2*COBWEB_MAX+2) # cobweb x-coordinates
cwy2 = np.zeros_like(cwx2) # cobweb y-coordinates

# Draw plot lines
@widgets.interact(step=(0, 2*COBWEB_MAX, 1), r=(0.5, 4.0, 0.01), x0=(0.0, 1.25, 0.01))
def update2(step=4, r=1.5, x0=0.65):
    global ax2, cwx2, cwy2
    
    # Cobweb plot
    ax2[0].clear()
    ax2[0].set_xlim([0, 1.5])
    ax2[0].set_ylim([0, 1.5])
    ax2[0].grid(False)
    ax2[0].set_title("Cobweb Plot")
    ax2[0].set_xlabel("$x_n$")
    ax2[0].set_ylabel("$x_{n+1}$")
    cobweb_update(x0, cwx2, cwy2, r=r, mode=1)
    ax2[0].plot(x, tmap(x, r=r), color="C0")
    ax2[0].plot(x, x, color="black")
    ax2[0].plot(cwx2[:step+2], cwy2[:step+2], color="C1")
    ax2[0].plot(cwx2[step:step+2], cwy2[step:step+2], color="red")
    
    # Scatter plot
    ax2[1].clear()
    ax2[1].set_ylim([0, 1.5])
    ax2[1].grid(False)
    ax2[1].set_title("Time Series")
    ax2[1].set_xlabel("$n$")
    ax2[1].set_ylabel("$x_n$")
    ax2[1].plot(np.append([0], nval[1:step+2:2]), np.append([x0], cwy2[1:step+2:2]), color="C0", marker=".", markersize=10)
    ax2[1].plot(nval[step+1], cwy2[step+1], color="red", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=4, description='step', max=40), FloatSlider(value=1.5, description='r', …